#**Covid19 Dashboard**

###This Dashboard plots new cases of Covid-19 by specimen date and the 7-day average of those cases against hospital admissions and new deaths within 28 days of a Covid-19 diagnosis.###

Please select desired area from the toggle box:

In [1]:
#The First step to running my dashboard is to perform the necessary imports.
#I'm importing a number of modules to process and plot the data, and ipywidgets to provide interactive controls.

from IPython.display import clear_output
import ipywidgets as wdg
from ipywidgets import HBox
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API
from datetime import datetime

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [3]:
#The next step is to import startup data:

#Initialise global variables to store the json data in. They are global so they can be updated later by other functions.

global cases_and_deaths
global eng_only
global sco_only
global wal_only
global ni_only

#load UK data and separate country data
with open("start_cases_and_deaths.json", "rt") as INFILE:
    cases_and_deaths=json.load(INFILE)
    
with open("eng_only_start_cases_and_deaths.json", "rt") as INFILE:
    eng_only = json.load(INFILE)
    
with open("sco_only_start_cases_and_deaths.json", "rt") as INFILE:
    sco_only = json.load(INFILE)

with open("wal_only_start_cases_and_deaths.json", "rt") as INFILE:
    wal_only = json.load(INFILE)
    
with open("ni_only_start_cases_and_deaths.json", "rt") as INFILE:
    ni_only = json.load(INFILE)


In [4]:
#Next, I will create my Data-Wrangling function

#A date parsing function, to turn string dates into Pandas date objects.
def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")

#The main data wrangling function.
#it creates a data frame and populates it with cases, hospitalisations, deaths and a 7-day average of cases.
def wrangle_cases_and_deaths_data(data):
    
    #extract the set of dicitonaries from the API response.
    datalist=data['data']
    
    #create a list of dates and sort it
    dates = [dictionary['date'] for dictionary in datalist]
    dates.sort()

    #create the date range
    start_date = parse_date(dates[0])
    end_date = parse_date(dates[-1])
    drange = pd.date_range(start_date,end_date,freq="D")
    
    #Create the empty dataframe with the date range and our desired column headings.
    timeseriesdf = pd.DataFrame(index=drange, columns = ["cases", "admissions", "deaths", "7 day average cases"])
    
    #adding case admission and death data
    for entry in datalist:
        date = parse_date(entry["date"])
        for column in ['cases', 'admissions', 'deaths']:
            #check that all dates have available data, and fill unavailable dates with 0.0
            if pd.isna(timeseriesdf.loc[date, column]):
                value = float(entry[column]) if entry[column]!= None else 0.0
                timeseriesdf.loc[date, column] = value
    
    #code to calculate 7 day average 
    #for the first 6 days, it tallies up total cases
    
    current_total = 0
    for i in range(len(datalist)):
        entry = datalist[i]
        cases = float(entry['cases']) if entry['cases'] != None else 0.0
        date = parse_date(entry["date"])
        if i <5:
            current_total += cases
#On the 7th day, it divides the total cases by 7 to get the average and saves it to the data frame.

        elif i == 6:
            current_total += cases
            current_av = current_total/7.0
            timeseriesdf.loc[date, "7 day average cases"] = current_av
#From then on, it deletes the case number from 7 days ago, adds a new one and recalculates and saves the average.
        else:
            del_entry = datalist[(i-7)]
            del_cases = float(del_entry['cases']) if del_entry['cases'] != None else 0.0
            current_total -= del_cases
            current_total += cases
            current_av = current_total/7.0
            timeseriesdf.loc[date, "7 day average cases"] = current_av
        i += 1
            
        
    return timeseriesdf # return the populated dataframe

In [5]:
#API accessing code - as my cases and deaths graphs are separated by nation,

# For the individual nations, I've created a get data function that takes area as a parameter and uses it in the filter call.

def get_area_data(area):
    filters = ["areaType=nation", f"areaName={area}"]
    structure={"date": "date",
                  "cases": "newCasesBySpecimenDate",
                  "admissions": "newAdmissions",
                  "deaths": "newDeaths28DaysByDeathDate"}
        
    api = Cov19API(filters=filters, structure=structure)
    data = api.get_json()
    return data # return data read from the API

#for the uk data, I've used "areaType=Overview"

def get_uk_data():
    
    filters = ["areaType=overview"]
    structure={"date": "date",
                  "cases": "newCasesBySpecimenDate",
                  "admissions": "newAdmissions",
                  "deaths": "newDeaths28DaysByDeathDate"}
    api = Cov19API(filters=filters, structure=structure)
    data = api.get_json()
    return data

#the access api function itself - calls the global variables from earlier and updates them using the functions I just defined.

def access_api():
    global cases_and_deaths
    global eng_only
    global sco_only
    global wal_only
    global ni_only
    
    cases_and_deaths = get_uk_data()
    eng_only = get_area_data("England")
    wal_only = get_area_data("Wales")
    sco_only = get_area_data("Scotland")
    ni_only = get_area_data("Northern Ireland")

In [6]:
#the button callback function
def api_button_callback(button):
    #update the json data, accounting for errors
    try:
        access_api()
        #This code updates the button label to show the time it was updated
        time = datetime.now()
        time = time.strftime("%H:%M:%S")
        button.description = f"data updated at: {time}"
        # refresh the graph using a 'trick' function, that switches the graph option to another option and back again.
        refresh_graph()
        # update the button icon and tooltip to account for the update having happened.
        apibutton.icon="check"
        button.tooltip=f"data updated at: {time}"
    except:
        print("update failed")

# Create my update button, with appropriate parameters.
apibutton=wdg.Button(
    description='Refresh Data', # you may want to change this...
    disabled=False,
    layout=wdg.Layout(width='200px'), # Changed the width to make sure whole time displays
    button_style='info',
    tooltip="Click to refresh data",
    icon="exclamation-triangle"
)

# registering callback function
apibutton.on_click(api_button_callback)


In [16]:
# function that returns desired JSON data from one of the global variables.

def get_data(area):
    if area == "UK":
        return cases_and_deaths
    if area == 'England':
        return eng_only
    if area == 'Wales':
        return wal_only
    if area == 'Scotland':
        return sco_only
    if area == 'Northern Ireland':
        return ni_only

# plotting function - recalculates the dataframe based on the fresh data and plots it
def plot_cases_and_deaths(area):
    jsondata = get_data(area)
    df = wrangle_cases_and_deaths_data(jsondata)    
    df.plot(figsize=(12,6)) #setting the size to get the x axis a bit longer,
    plt.legend() 
    plt.show()

# Graph refreshing function - uses a trick, in that it switches
# the graph to a different panel and then switches back to the original
# to force a redraw.

def refresh_graph():
    current = whicharea.value
    other = whicharea.options[1]
    if current == whicharea.options[1]:
        other = whicharea.options[0]
    else:
        other == whicharea.options[1]
    whicharea.value = other
    whicharea.value = current

# creating the toggle buttons, with a label for each country/area.
whicharea=wdg.ToggleButtons(
    options=['UK', 'England', 'Wales', 'Scotland', 'Northern Ireland'],
    value='UK',
    description='Area: ',
    disabled=False,
)

# placing the toggles in an HBox to get them to display horizontally
toggle_box = HBox([whicharea])

# using the interactive output method to pass the value of the toggle button into the plot_cases_and_deaths function
graph=wdg.interactive_output(plot_cases_and_deaths, {'area': whicharea})

# displaying the toggles and graph
display(toggle_box, graph)

Output()

Initial data downloaded on 5/12/21. 
Please click the button below to update the graph with the latest available data.

In [8]:
#api button displayed down here to allow the markdown box above!
display(apibutton)

Button(button_style='info', description='Refresh Data', icon='exclamation-triangle', layout=Layout(width='200p…

_DIY Covid-19 Dashboard (C) Harry Evans 2021 (ec21939@qmul.ac.uk)._
_Based on UK Government data published by Public Health England Template by Dr. Fabrizio Smeraldi_